## 1 1. Inicialización

In [2]:
#importar librerías
import pandas as pd
import matplotlib as plt

## 2 1.2 Cargar datos

In [4]:
#cargar datasets
df_full = pd.read_csv('files/data gold/gold_recovery_full.csv')
df_test = pd.read_csv('files/data gold/gold_recovery_test.csv')
df_train = pd.read_csv('files/data gold/gold_recovery_train.csv')